In [24]:
import os
import paramiko
import pandas as pd
import time
import pycurl
from selenium import webdriver
import certifi
from io import BytesIO
import urllib3
import json
from rich.console import Console
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys


In [25]:
confs = open("../config.json")
confs = json.load(confs)

server = confs["server"]
port = confs["port"]
username = confs["username"]
password = confs["password"]
DeployBranch = confs["DeployBranch"]
githubKey = confs["githubKey"]
RepoLink = confs["RepoLink"]
domain = confs["domain"]
RootUrl = confs["RootUrl"]
sudomains = confs["sudomains"]
folderName = RepoLink.split("/")[4].replace(".git", "")
repoServerpath = ''
clientPath = "" if confs["clientPath"] == "/" else confs["clientPath"]
console=Console()

In [26]:
#https://leila.leila.care/
#93.92.134.143
#https://prelive.leila.de
#https://webapp.leila.de
# p1Zz09#

SeleniumData = [
    ['https://example.com/',
    'in?|Domain'],
    ['https://lgtest.Kaiserfranz-engineering.de',
    'in?|welcome to lgtest'],
    ['https://aottest.Kaiserfranz-engineering.de',
    'in?|welcome to aottest'],
    ['https://api.kaiserfranz-engineering.de/',
    'in?|test'],
    ['https://api.kaiserfranz-engineering.de/api',
    'in?|models']
]

In [29]:

#NOTE We use selenum, because it is present on all windows machines with the same path.
#NOTE IMPORTANT: use selenium version 4 and upwards

driver = webdriver.Edge(executable_path = r"./msedgedriver.exe") # Modify the path here...


def ExecuteSeleniumTests(actions):
    '''This function takes an array of arrays of strings. The Strings are seperated by | and mapped onto selenium dirver actions.'''
    for data in actions:
        for action in data:
            try:
                if 'http' in action:
                    print(f'gettting {action}')
                    driver.get(action)
                elif 'present?' in action:

                    print(f'checking if element is present{action}')
                    args = action.split('|')
                    webEl  = driver.find_element_by_xpath(f"//{args[2]}[@{args[0]}='{args[1]}']")
                    if webEl.is_displayed():
                        console.print(f'Test for {data[0]} returns: SUCCESS', style='bold green on white', justify='left')
                        continue
                elif 'xpath' in action: 
                    print(f'clicking with xpath{action}')
                    el= driver.find_element_by_xpath(action.split('|')[0])
                    if action.split('|')[2] == 'click':
                        el.click()

                elif action.count('in?'):
                    print(f'test for site containing text ')
                    text = action.split('|')[1] 
                    print(f'site should contain {text}')
                    if text in driver.page_source:
                        console.print(f'Test for {data[0]} returns: SUCCESS', style='bold green on white', justify='left')
                        continue
                elif 'wait?' in action: 
                    print('waiting')
                    time.sleep(int(action.split('|')[0]))

                elif 'submit?' in action: 
                    print('submitting')
                    time.sleep(1)
                    webEl  = driver.find_element_by_xpath(f"//{args[2]}[@{args[0]}='{args[1]}']")
                    webEl.submit()
                elif 'url?' in action: 
                    print(f'checking for url match{action}')
                    time.sleep(4)
                    print(action)
                    args = action.split('|')
                    print(args[0] + ' ' + driver.current_url)
                    if args[0] == driver.current_url:
                        console.print(f'Test for {data[0]} returns: SUCCESS', style='bold green on white', justify='left')
                        continue
                else:
                    print('locating element')
                    args = action.split('|')
                    webEl  = driver.find_element_by_xpath(f"//{args[2]}[@{args[0]}='{args[1]}']")
                    if args[3] =='click':
                        driver.implicitly_wait(1)
                        webEl.click()
                    else:
                            webEl.send_keys(args[3])
            
            except Exception as e:
                console.print(f'Test for {data[0]} returns: Failed', style='bold red on white', justify='left')
                print(action)
                print(e.args)
                continue
    driver.close()

ExecuteSeleniumTests(SeleniumData)

C:\Users\franz\AppData\Local\Temp/ipykernel_15276/4215595938.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(executable_path = r"./msedgedriver.exe") # Modify the path here...


gettting https://example.com/
test for site containing text 
site should contain Domain


Test for https://example.com/ returns: SUCCESS                                                                     

gettting https://lgtest.Kaiserfranz-engineering.de


Test for https://lgtest.Kaiserfranz-engineering.de returns: Failed                                                 

...

https://aottest.Kaiserfranz-engineering.de
()
test for site containing text 
site should contain welcome to aottest
gettting https://api.kaiserfranz-engineering.de/


Test for https://api.kaiserfranz-engineering.de/ returns: Failed                                                   

https://api.kaiserfranz-engineering.de/
()
test for site containing text 
site should contain test
gettting https://api.kaiserfranz-engineering.de/api


Test for https://api.kaiserfranz-engineering.de/api returns: Failed                                                

https://api.kaiserfranz-engineering.de/api
()
test for site containing text 
site should contain models
